# TEST

In [1]:
#%pip install openneuro-py mne mne_bids naplib autoreject PyWavelets --quiet

In [2]:
import os
import numpy as np
import mne
from mne.datasets import sample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from openneuro import download
from autoreject import get_rejection_threshold

/Users/evakhromeeva/Projects/BeSMPSegmentation/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Загрузка данных
# download(dataset='ds002778', target_dir='./ds002778')

In [4]:
# Расположение данных
dataset_path = '/Users/evakhromeeva/mne_data/ds002778'

In [5]:
# Параметры датасета
dataset = "ds002778"
subjects_pd = ["sub-pd3", "sub-pd5", "sub-pd6", "sub-pd6", "sub-pd9","sub-pd11", "sub-pd12", "sub-pd13", "sub-pd14", "sub-pd16", "sub-pd17", "sub-pd19", "sub-pd22", "sub-pd23", "sub-pd26", "sub-pd28"]# subject with PD
subjects_hc = ["sub-hc1", "sub-hc2", "sub-hc4", "sub-hc7", "sub-hc8", "sub-hc10", "sub-hc18", "sub-hc20", "sub-hc21", "sub-hc24", "sub-hc25", "sub-hc29", "sub-hc30", "sub-hc31", "sub-hc32", "sub-hc33"] # healthy subject

In [6]:
# путь до файла с ЭЭГ выбранного пациента (например, sub-pd6)
def get_eeg_data_path(sub):
    return f'{dataset_path}/{sub}/ses-off/eeg/{sub}_ses-off_task-rest_eeg.bdf' if sub.startswith('sub-pd') else f'{dataset_path}/{sub}/ses-hc/eeg/{sub}_ses-hc_task-rest_eeg.bdf'

In [7]:
# Загрузка и предобработка данных
def load_and_preprocess_data(data_list, is_healthy=True, duration=2.0):
    raw_files = [get_eeg_data_path(f) for f in data_list]
    epochs_list = []
    labels = []

    for raw_file in raw_files:
        raw = mne.io.read_raw_bdf(raw_file, preload=True)
        raw.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4','EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']) #drop extra channels
        raw.set_eeg_reference(ref_channels='average')
        raw.filter(0.5, None, fir_design='firwin',phase='zero-double') #remove drifts
        event_id = {'healthy': 0, 'parkinson': 1}
        epochs = mne.make_fixed_length_epochs(raw, duration=duration, preload=False, proj=True)
        reject = get_rejection_threshold(epochs)
        epochs.drop_bad(reject=reject)
        epochs_list.append(epochs)
        labels.extend([event_id['healthy'] if is_healthy else event_id['parkinson']] * len(epochs))

    epochs_data = np.concatenate([e.get_data() for e in epochs_list])
    labels = np.array(labels)
    return epochs_data, labels

In [19]:
# Преобразование данных для PyTorch
def prepare_torch_data(X, y):
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X, y)
    return dataset

In [18]:
# Определение модели нейронной сети
import torch
import torch.nn as nn

class ImprovedEEGNet(nn.Module):
    def __init__(self, n_classes=2):
        super(ImprovedEEGNet, self).__init__()
        
        # Сверточные слои
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(1, 64), padding=(0, 32))
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(32, 1), groups=32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(1, 32), padding=(0, 16))
        self.batchnorm3 = nn.BatchNorm2d(128)
        
        # Пулинг
        self.pooling = nn.AvgPool2d(kernel_size=(1, 8))
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
        # Рекуррентный слой
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)
        
        # Полносвязные слои
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = torch.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = torch.relu(self.conv3(x))
        x = self.batchnorm3(x)
        x = self.pooling(x)
        x = self.dropout(x)
        
        # Подготовка данных для LSTM
        x = x.squeeze(2)  # Удаляем размерность каналов
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, features)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Используем последний выход LSTM
        
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
# Обучение модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [21]:
# Основной пайплайн
X_pd, y_pd = load_and_preprocess_data(subjects_pd, is_healthy=False)
X_hc, y_hc = load_and_preprocess_data(subjects_hc, is_healthy=True)

Extracting EDF parameters from /Users/evakhromeeva/mne_data/ds002778/sub-pd3/ses-off/eeg/sub-pd3_ses-off_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 102399  =      0.000 ...   199.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.5 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.25 Hz)
- Filter length: 3381 samples (6.604 s)

Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 100 events

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Using data from preloaded Raw for 100 events and 1024 original time points ...
    Rejecting  epoch based on EEG : ['FC5']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'T7', 'CP5', 'T8', 'FC6']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['F7', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF4', 'Fp2']
9 bad epochs dropped
Extracting EDF parameters from /Users/evakhromeeva/mne_data/ds002778/sub-pd5/ses-off/eeg/sub-pd5_ses-off_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 100863  =      0.000 ...   196.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pa

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Using data from preloaded Raw for 98 events and 1024 original time points ...
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC6', 'Fp2']
    Rejecting  epoch based on EEG : ['AF4', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'F7', 'FC6', 'F8', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'F4', 'AF4', 'Fp2', 'Fz']
 

...

Using data from preloaded Raw for 93 events and 1024 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'O1', 'Fp2']
    Rejecting  epoch based on EEG : ['O1']
    Rejecting  epoch based on EEG : ['Fp1', 'F7', 'F3', 'FC5', 'T7', 'C3', 'CP5', 'O1', 'P8', 'CP6', 'C4', 'T8', 'FC6', 'F8', 'Fp2']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'T7', 'O1', 'P8', 'CP6', 'C4', 'T8', 'FC6', 'F8']
    Rejecting  epoch based on EEG : ['T8']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'T7', 'O1', 'P8', 'CP6', 'T8', 'FC6']
    Rejecting  epoch based on EEG : ['Fp1', 'F7', 'F3', 'FC5', 'T7', 'C3', 'CP5', 'O1', 'P8', 'CP6', 'C4', 'T8', 'FC6', 'F8', 'Fp2']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'T7', 'CP5', 'CP6', 'T8', 'FC6']
    Rejecting  epoch based on EEG : ['O1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'C3', 'CP5', 'P7', 'P4', 'P8', 'CP6', 'C4', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Using data from preloaded Raw for 98 events and 1024 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'AF4', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'F7', 'F3', 'T7', 'O2', 'PO4', 'P8', 'T8', 'AF4', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'Fp2']
12 bad epochs dropped
Extracting EDF parameters from /Users/evakhromeeva/mne_data/ds002778/sub-hc33/ses-hc/eeg/sub-hc33_ses-hc_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structu

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Using data from preloaded Raw for 95 events and 1024 original time points ...
    Rejecting  epoch based on EEG : ['F3']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'AF3', 'F7', 'F3', 'FC5', 'F8', 'Fp2']
3 bad epochs dropped
Using data from preloaded Raw for 58 events and 1024 original time points ...
Using data from preloaded Raw for 20 events and 1024 original time points ...
Using data from preloaded Raw for 90 events and 1024 original time points ...
Using data from preloaded Raw for 77 events and 1024 original time points ...
Using data from preloaded Raw for 86 events and 1024 original time points ...
Using data from preloaded Raw for 61 events and 1024 original time points ...
Using data from preloaded Raw for 75 events and 1024 original time points ...
Using data from preloaded Raw for 92 events and 1024 original time points ...
Using data from preloaded Raw for 38 events and 1024 original time points ...
Using data from preloa

In [22]:
# Объединение данных больных и здоровых в единый тензор
print(X_pd.shape)
print(X_hc.shape)
print(y_pd.shape)
print(y_hc.shape)
X = np.concatenate((X_pd, X_hc), axis=0)
y = np.concatenate((y_pd, y_hc), axis=0)
print(X.shape)
print(y.shape)


(1175, 32, 1024)
(1088, 32, 1024)
(1175,)
(1088,)
(2263, 32, 1024)
(2263,)


In [23]:
# Разделение на тренировночную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Преобразование данных для PyTorch
train_dataset = prepare_torch_data(X_train[:, np.newaxis, :, :], y_train)
test_dataset = prepare_torch_data(X_test[:, np.newaxis, :, :], y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [26]:
# Инициализация модели, функции потерь и оптимизатора
model = ImprovedEEGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
# Обучение модели
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.5688149929046631
Epoch 2/10, Loss: 0.703802227973938
Epoch 3/10, Loss: 0.36680468916893005
Epoch 4/10, Loss: 0.5223544239997864
Epoch 5/10, Loss: 0.5086537599563599
Epoch 6/10, Loss: 0.45140540599823
Epoch 7/10, Loss: 0.5659947991371155
Epoch 8/10, Loss: 0.6773388385772705
Epoch 9/10, Loss: 0.15382784605026245
Epoch 10/10, Loss: 0.4849339723587036


In [28]:
# Оценка модели
model.eval()
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())

print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

              precision    recall  f1-score   support

           0       0.48      1.00      0.65       216
           1       0.00      0.00      0.00       237

    accuracy                           0.48       453
   macro avg       0.24      0.50      0.32       453
weighted avg       0.23      0.48      0.31       453

Accuracy: 0.4768211920529801


/Users/evakhromeeva/Projects/BeSMPSegmentation/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evakhromeeva/Projects/BeSMPSegmentation/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evakhromeeva/Projects/BeSMPSegmentation/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh